# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np
import scipy
import math
from math import sqrt
import os
import tensorflow as ts
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array 
from collections import Counter
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import math
from collections import Counter
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.probability import FreqDist
from sklearn.metrics import mean_squared_error

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Importing SICK dataset 

In [ ]:
import requests

def download_sick(f): 

    response = requests.get(f).text

    lines = response.split("\n")[1:]
    lines = [l.split("\t") for l in lines if len(l) > 0]
    lines = [l for l in lines if len(l) == 5]

    df = pd.DataFrame(lines, columns=["idx", "sent_1", "sent_2", "sim", "label"])
    df['sim'] = pd.to_numeric(df['sim'])
    return df
    
sick_train = download_sick("https://raw.githubusercontent.com/alvations/stasis/master/SICK-data/SICK_train.txt")
sick_dev = download_sick("https://raw.githubusercontent.com/alvations/stasis/master/SICK-data/SICK_trial.txt")
sick_test = download_sick("https://raw.githubusercontent.com/alvations/stasis/master/SICK-data/SICK_test_annotated.txt")
sick_all = sick_train.append(sick_test).append(sick_dev)


In [ ]:
sick_test

,idx,sent_1,sent_2,sim,label
0,6,There is no boy playing outdoors and there is ...,A group of kids is playing in a yard and an ol...,0.57500,NEUTRAL\r
1,7,A group of boys in a yard is playing and a man...,The young boys are playing outdoors and the ma...,0.67500,NEUTRAL\r
2,8,A group of children is playing in the house an...,The young boys are playing outdoors and the ma...,0.50000,NEUTRAL\r
3,10,A brown dog is attacking another animal in fro...,A brown dog is attacking another animal in fro...,0.97500,ENTAILMENT\r
4,11,A brown dog is attacking another animal in fro...,A brown dog is helping another animal in front...,0.66625,NEUTRAL\r
...,...,...,...,...,...
4922,9991,The young girl is blowing a bubble that is huge,There is no girl in pink twirling a ribbon,0.27500,NEUTRAL\r
4923,9992,A dog in a colored coat is running across the ...,The flute is being played by one man,0.00000,NEUTRAL\r
4924,9994,A boy is happily playing the piano,A white bird is landing swiftly in the water,0.00000,NEUTRAL\r
4925,9995,"The girl, who is little, is combing her hair i...",Two people wearing helmets are driving over th...,0.00000,NEUTRAL\r


In [ ]:
print(sick_train[:20])

   idx                                             sent_1  ...    sim          label
0    1  A group of kids is playing in a yard and an ol...  ...  4.500        NEUTRAL
1    2  A group of children is playing in the house an...  ...  3.200        NEUTRAL
2    3  The young boys are playing outdoors and the ma...  ...  4.700     ENTAILMENT
3    5  The kids are playing outdoors near a man with ...  ...  3.400        NEUTRAL
4    9  The young boys are playing outdoors and the ma...  ...  3.700        NEUTRAL
5   12                              Two dogs are fighting  ...  4.000        NEUTRAL
6   14  A brown dog is attacking another animal in fro...  ...  3.500        NEUTRAL
7   18  A brown dog is attacking another animal in fro...  ...  3.200        NEUTRAL
8   25          Nobody is riding the bicycle on one wheel  ...  2.800        NEUTRAL
9   26        A person is riding the bicycle on one wheel  ...  3.700        NEUTRAL
10  28  A person on a black motorbike is doing tricks ...  ...  3

# Normalization of "sim" column

In [ ]:
def normalize(df, feature_names):
    result = df.copy()
    for feature_name in feature_names:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
sick_all = normalize(sick_all,['sim'])
sick_test = normalize(sick_test,['sim'])
sick_train = normalize(sick_train,['sim'])
sick_dev = normalize(sick_dev,['sim'])

# Preprocessing of lower casing

In [ ]:
# sick_train['sent_1'] = sick_train['sent_1'].str.lower()
# sick_train['sent_2'] = sick_train['sent_2'].str.lower()

In [ ]:
df1 = sick_test['sent_1'].values
# df1 = ["Cat, Dog are pet animals."]

In [ ]:
df2 = sick_test['sent_2'].values
# df2 = ["Cat, Dog and Birds are pet animals."]

In [ ]:
print(len(df1),len(df2))

4927 4927


In [ ]:
sick_train[sick_train["sim"]<0.5].count()

idx       1071
sent_1    1071
sent_2    1071
sim       1071
label     1071
dtype: int64

In [ ]:
sick_train[sick_train["sim"]>0.5].count()

idx       3299
sent_1    3299
sent_2    3299
sim       3299
label     3299
dtype: int64

In [ ]:
sick_train[sick_train["sim"]>0.8].count()

idx       1216
sent_1    1216
sent_2    1216
sim       1216
label     1216
dtype: int64

In [ ]:
sick_train["sim"].value_counts()

0.62500    205
0.67500    204
0.97500    196
0.65000    194
0.60000    185
          ... 
0.22125      1
0.95875      1
0.63750      1
0.38375      1
0.43375      1
Name: sim, Length: 125, dtype: int64

In [ ]:
list(sick_train["sim"].value_counts())

[205,
 204,
 196,
 194,
 185,
 174,
 172,
 169,
 164,
 159,
 157,
 156,
 153,
 153,
 147,
 146,
 141,
 135,
 130,
 125,
 117,
 115,
 105,
 90,
 74,
 71,
 64,
 62,
 59,
 48,
 33,
 31,
 30,
 29,
 29,
 26,
 26,
 25,
 25,
 25,
 24,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
sick_test["sim"].value_counts()

0.65000    231
0.70000    215
0.60000    215
0.62500    207
0.97500    203
          ... 
0.93375      1
0.62875      1
0.47875      1
0.81750      1
0.67875      1
Name: sim, Length: 146, dtype: int64

In [ ]:
sick_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   idx     4500 non-null   object 
 1   sent_1  4500 non-null   object 
 2   sent_2  4500 non-null   object 
 3   sim     4500 non-null   float64
 4   label   4500 non-null   object 
dtypes: float64(1), object(4)
memory usage: 175.9+ KB


In [ ]:
import numpy as np
df = np.concatenate((df1,df2), axis=0)
df

array(['There is no boy playing outdoors and there is no man smiling',
       'A group of boys in a yard is playing and a man is standing in the background',
       'A group of children is playing in the house and there is no man standing in the background',
       ..., 'A white bird is landing swiftly in the water',
       'Two people wearing helmets are driving over the yellow and white flowers',
       'The snowboarder is leaping fearlessly over white snow'],
      dtype=object)

In [ ]:
# df=df1+" "+df2
# len(df)

In [ ]:
d_string = ''
for i in range(len(df)):
  s1 = df[i]
  d_string += ' '+(s1) 
#print(d1_string)

In [ ]:
d_string

" There is no boy playing outdoors and there is no man smiling A group of boys in a yard is playing and a man is standing in the background A group of children is playing in the house and there is no man standing in the background A brown dog is attacking another animal in front of the tall man in pants A brown dog is attacking another animal in front of the man in pants Two dogs are wrestling and hugging A brown dog is attacking another animal in front of the man in pants Two dogs are wrestling and hugging Two dogs are wrestling and hugging A person in a black jacket is doing tricks on a motorbike There is no man in a black jacket doing tricks on a motorbike A person in a black jacket is doing tricks on a motorbike A skilled person is riding a bicycle on one wheel Nobody is riding the bicycle on one wheel A person is riding the bicycle on one wheel A person is riding the bicycle on one wheel A man with a jersey is dunking the ball at a basketball game The game of basketball consists o

# Tokenization

In [ ]:
#tokens_TF_idf=nltk.wordpunct_tokenize(d_string)

In [ ]:
#len(tokens_TF_idf)

# Preprocessing Text

In [ ]:
def preprocess(text):
     
      updated = re.sub('<[^<]+?>', '', text) # html tags
      updated = re.sub(r'\W', ' ', str(text)) # Remove all the special characters
      updated = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters
      updated = re.sub(r'\^[a-zA-Z]\s+', ' ', text) # Remove single characters from the start
      updated = re.sub(r'\s+', ' ', text, flags=re.I) # substituting multiple spaces with single space
      updated = re.sub(r'^b\s+', '', text) # removing prefixed b 
      table = str.maketrans({key: None for key in string.punctuation})
      updated = text.translate(table)
      return updated

In [ ]:
import re

#d_string = [preprocess(sent) for sent in df]

d_string

" There is no boy playing outdoors and there is no man smiling A group of boys in a yard is playing and a man is standing in the background A group of children is playing in the house and there is no man standing in the background A brown dog is attacking another animal in front of the tall man in pants A brown dog is attacking another animal in front of the man in pants Two dogs are wrestling and hugging A brown dog is attacking another animal in front of the man in pants Two dogs are wrestling and hugging Two dogs are wrestling and hugging A person in a black jacket is doing tricks on a motorbike There is no man in a black jacket doing tricks on a motorbike A person in a black jacket is doing tricks on a motorbike A skilled person is riding a bicycle on one wheel Nobody is riding the bicycle on one wheel A person is riding the bicycle on one wheel A person is riding the bicycle on one wheel A man with a jersey is dunking the ball at a basketball game The game of basketball consists o

# Lemmatization

In [ ]:
# def lemmatize(d1_tokens):
#     lz = WordNetLemmatizer()
#     lemmatized = [lz.lemmatize(t) for t in d1_tokens]
#     return list(set(lemmatized))

# StopWords

In [ ]:
en_stop = set(nltk.corpus.stopwords.words('english'))
# from nltk.stem import WordNetLemmatizer
# stemmer = WordNetLemmatizer()

# Tokenization

In [ ]:
#d_string=tokens_TF_idf

In [ ]:
tokens1 = [sent.split() for sent in df1]
for i in range(len(tokens1)):
  # tokens1[i] = [stemmer.lemmatize(word) for word in tokens1[i]]
  #tokens[i] = [word for word in tokens[i] if word not in en_stop]
  tokens1[i] = [word for word in tokens1[i] if len(word) > 2]

In [ ]:
tokens2 = [sent.split() for sent in df2]
for i in range(len(tokens2)):
  # tokens2[i] = [stemmer.lemmatize(word) for word in tokens2[i]]
  #tokens[i] = [word for word in tokens[i] if word not in en_stop]
  tokens2[i] = [word for word in tokens2[i] if len(word) > 2]

In [ ]:
tokens1_with_stop = tokens1
#tokens_without_stop = tokens
# tokens1_with_stop

In [ ]:
tokens2_with_stop = tokens2
#tokens_without_stop = tokens
# tokens2_with_stop

In [ ]:
#d1_lemmas = lemmatize(tokens_with_stop)
#len(d1_lemmas)

In [ ]:
#d2_lemmas = lemmatize(tokens_without_stop)
#len(d2_lemmas)

In [ ]:
STOP = set(nltk.corpus.stopwords.words("english"))

In [ ]:
import nltk
class SentenceModel:

    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        self.tokens = [t for t in nltk.word_tokenize(normalized_sentence)]
        self.tokens_without_stop = [t for t in self.tokens if t not in STOP]



# Cosine Similarity

In [ ]:
def run_cosine_similarity(sentences1, sentences2, model=None, use_stoplist=False, doc_freqs=None):
    if doc_freqs is not None:
        N = doc_freqs["NUM_DOCS"]

    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):

        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens
        
        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        if len(tokens1) == 0 or len(tokens2) == 0:
            sims.append(0)
            continue

        tokfreqs1 = Counter(tokens1)
        tokfreqs2 = Counter(tokens2)

        weights1 = [tokfreqs1[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs1] if doc_freqs else None
        weights2 = [tokfreqs2[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs2] if doc_freqs else None

        embedding1 = np.average([model[token] for token in tokfreqs1], axis=0, weights=weights1).reshape(1, -1)
        embedding2 = np.average([model[token] for token in tokfreqs2], axis=0, weights=weights2).reshape(1, -1)

        sim = cosine_similarity(embedding1, embedding2)[0][0]
        sims.append(sim)

    return sims
    

# Word Mover's Distance

In [ ]:
from sklearn import preprocessing
def run_wmd_similarity(sentences1, sentences2, model, use_stoplist=False):
    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):

        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        if len(tokens1) == 0 or len(tokens2) == 0:
            tokens1 = [token for token in sent1.tokens if token in model]
            tokens2 = [token for token in sent2.tokens if token in model]

        sims.append(-model.wmdistance(tokens1, tokens2))
    
    
    x_array = np.array(sims)
    print(x_array)
    normalized_arr = preprocessing.normalize([x_array])
    sims=normalized_arr[0].tolist()
    return sims

    

# Smooth Inverse Frequency

In [ ]:
list1=[]
[list1.append(i) for sent_1 in tokens1_with_stop for i in sent_1]
print(len(list1))

[list1.append(i) for sent_2 in tokens2_with_stop for i in sent_2]
print(len(list1))
frequencies = FreqDist(list1)


32187
63865


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    print(X)
    print(type(X))
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX


def run_sif_similarity(sentences1, sentences2, model, freqs={}, use_stoplist=False, a=1):
    total_freq = sum(freqs.values())

    embeddings = []

    # SIF 

    for (sent1, sent2) in zip(sentences1, sentences2):
        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        weights1 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens1]
        weights2 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens2]

        embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
        embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)

        embeddings.append(embedding1)
        embeddings.append(embedding2)

    embeddings = remove_first_principal_component(np.array(embeddings))
    sims = [cosine_similarity(embeddings[idx * 2].reshape(1, -1),
                              embeddings[idx * 2 + 1].reshape(1, -1))[0][0]
            for idx in range(int(len(embeddings) / 2))]

    return sims
                

# Google News Pre-Trained Word2Vec Model Implementation

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-02-05 06:38:02--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.142.222
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.142.222|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'

In [ ]:
import gensim

In [ ]:
google_pretrained_model = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# Evaluation metrics

In [ ]:
sentences1 = [SentenceModel(s) for s in sick_test['sent_1']] 
sentences2 = [SentenceModel(s) for s in sick_test['sent_2']]
#sentences1 = [SentenceModel(s) for s in df1] 
#sentences2 = [SentenceModel(s) for s in df2]
#sims = run_cosine_similarity(sentences1,sentences2, model=google_pretrained_model, use_stoplist=True) # cosine 
#sims = run_wmd_similarity(sentences1,sentences2, model=google_pretrained_model, use_stoplist=True) #word mover distance
sims = run_sif_similarity(sentences1,sentences2, freqs=frequencies, model=google_pretrained_model, use_stoplist=True)
#sims = run_context_avg_benchmark(sentences1, sentences2, model=elmo, use_stoplist=False, doc_freqs=None)

sick_test["predicted_sim"] = pd.Series(sims).values
sick_test["diff"] = np.sqrt((sick_test["predicted_sim"] - sick_test["sim"])**2)
# sick_train = sick_train.sort_values("diff", ascending=False)
pearson_correlation = scipy.stats.pearsonr(sims, sick_test['sim'])[0]
spearman_correlation = scipy.stats.spearmanr(sims, sick_test['sim'])[0]
mse = (mean_squared_error(sims, sick_test['sim']))
textstr = 'MSE=%.3f\nPearson Correlation=%.3f\nSpearman Correlation=%.3f'%(mse, pearson_correlation, spearman_correlation)
sick_test['predicted_sim'] = pd.Series(sims).values
sick_test = sick_test.sort_values('sim')
id = list(range(0, len(sick_test.index)))
sick_test['id'] = pd.Series(id).values
    

[[ 0.02985052  0.12298476  0.02312861 ... -0.07786941 -0.04850036
  -0.05911921]
 [ 0.03840546 -0.06304366  0.01040092 ...  0.10614869  0.05871932
  -0.0642961 ]
 [ 0.07144727  0.04093062 -0.04749341 ...  0.11173266  0.12722696
   0.05333685]
 ...
 [ 0.09136742  0.00487469  0.11317859 ... -0.09609876  0.0005932
  -0.08519326]
 [ 0.02836397  0.10195507 -0.01969968 ...  0.18282047  0.00557072
   0.04702653]
 [ 0.0194774   0.11072394 -0.0218959  ...  0.18538859  0.02044976
   0.02662754]]
<class 'numpy.ndarray'>


In [ ]:
sick_train

,idx,sent_1,sent_2,sim,label,predicted_sim,diff,id
4499,10000,Three dogs are resting on a sidewalk,The woman with a knife is slicing a pepper,0.0,NEUTRAL,-0.032835,0.032835,0
2110,4763,A person is dancing in the rain,A man is singing into a microphone,0.0,NEUTRAL,-0.020598,0.020598,1
2119,4788,An elephant is being ridden by a woman,A woman is opening a soda and drinking it,0.0,NEUTRAL,-0.025649,0.025649,2
4422,9825,The girl is carrying a sign and a group of peo...,A woman is cleaning a man's face,0.0,NEUTRAL,-0.023332,0.023332,3
573,1317,A person is performing a card trick,A man is frying a tortilla,0.0,NEUTRAL,-0.025593,0.025593,4
...,...,...,...,...,...,...,...,...
2899,6395,A dog is herding a group of white sheep on a d...,A dog is herding a pack of white sheep on a du...,1.0,ENTAILMENT,-0.003057,1.003057,4495
2891,6377,A surfer is leaning on a surfboard,The surfer is leaning on a surfboard,1.0,ENTAILMENT,-0.004192,1.004192,4496
952,2136,A woman is peeling a potato,A potato is being peeled by a woman,1.0,ENTAILMENT,-0.005274,1.005274,4497
2902,6404,"The male, who is dark skinned, is standing on ...",The dark skinned male is standing on one hand ...,1.0,ENTAILMENT,-0.000000,1.000000,4498


In [ ]:
# cosine
print(textstr)

MSE=0.041
Pearson Correlation=0.727
Spearman Correlation=0.621
